(C) Leya Zahra Akhtar, 2020  All rights reserved.

# Covid-19 Dashboard

This is a dashboard displaying data for COVID-19 in England.
The graph below shows statistics regarding the effectiveness of COVID-19 vaccinations over the course of the year.

It is clear that there is a correlation so far between the number of people receiving their first and second doses and the number of daily new cases and deaths slowly levelling off by autumn 2021. Moreover, the transmissabiltiy of the virus has not peaked as high as earlier in 2020. This clearly shows that vaccination rollouts have been effective across England. Therefore, booster vaccines should be considered in the future to lower the number of cases and deaths further to maintain these figures.

The graph can be interacted with by clicking SHIFT to choose which statistics you wish to view. You can select multiple statistics at one time. Users can also switch between linear and log scales. (Log scale is recommended).

In [29]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [30]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [31]:

#Load JSON files and store the raw data in a variable called data
with open('cases.json') as case:
    data = json.load(case)
#data #initial data that will be presented on the dashboard before it is refreshed to fetch updated API data


In [32]:
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")


In [33]:
#wrangling the data so it is organised and ready to be plotted correctly 
def wrangle_data(data):
    datalist=data['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    casesdata=pd.DataFrame(index=index, columns=['new cases', 'new deaths', 'total vaccinated 1st dose','total vaccinated 2nd dose','transmissability - R'])
    for entry in datalist: 
        date=parse_date(entry['date'])
        for column in ['new cases', 'new deaths', 'total vaccinated 1st dose','total vaccinated 2nd dose','transmissability - R']:
            if pd.isna(casesdata.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                casesdata.loc[date, column]=value
            

    casesdata.fillna(0.0, inplace=True)
    return casesdata
casesdata=wrangle_data(data)


In [34]:
#API access code which will be called when the button is pressed by the user. Data taken from PHE website
def access_api():
    filters = [
    'areaType=nation',
    'areaName=england'
]



    structure = {
    "date":"date",
    "new cases": "newCasesByPublishDate",
    "new deaths":"newDeaths28DaysByDeathDate",
    "total vaccinated 1st dose":"cumPeopleVaccinatedFirstDoseByVaccinationDate",
    "total vaccinated 2nd dose":"cumPeopleVaccinatedSecondDoseByVaccinationDate",
    "transmissability - R":"transmissionRateMax"
    
}
    api = Cov19API(filters=filters, structure=structure)
    cases=api.get_json()
    #print(cases)
  
    
    
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    return cases # return data read from the API


In [35]:

def api_button_callback(apibutton):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # This will get new data from my API
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global casesdata
    casesdata=wrangle_data(apidata)
    # When the user wants updated data, they can click the button
    refresh_graph()
    # Once the data has been updated once, it does not really need to be updated again immediately.
    #Therefore, the api button is disabled once it has been clicked once.
    apibutton.icon="check"
    apibutton.disabled=True

#This is my widget that will refresh the data once it has been pressed   
apibutton=wdg.Button(
    description='Update data',
    disabled=False,
    button_style='info', 
    tooltip='Click to access updated Government COVID-19 data',
    icon='refresh' 
    
)


apibutton.on_click(api_button_callback)

display(apibutton)

Button(button_style='info', description='Update data', icon='refresh', style=ButtonStyle(), tooltip='Click to …

In [36]:

#A widget that allows the user to select more than one category of data at a time by clicking SHIFT

series=wdg.SelectMultiple(
    options=['new cases', 'new deaths', 'total vaccinated 1st dose','total vaccinated 2nd dose','transmissability - R'],
    value=['new cases', 'new deaths', 'total vaccinated 1st dose','total vaccinated 2nd dose','transmissability - R'],
    rows=5,
    description='View by:',
    disabled=False,
    

    
)


#A widget that allows the user to switch between linear and logarithm.
#I changed the button style to info to match better with the colour scheme of my graph

scale= wdg.ToggleButtons(
    options =['linear','log'],
    description='Scale: ',
    disabled =False,
    button_style='info'
   
)

#Experimenting with inserting images and adding a PHE logo alongside the data

"""file = open('PHE.jpg','rb')
image = file.read()
logo=wdg.Image(
    value =image,
    format ='jpg',
    width = 100,
    height = 40,)"""



#If the number of columns is more than 0, display the graph. If it is 0, tell the user to select one or more category to view.

def cases_graph(column, scale):
    if scale=='linear':
        logscale=False
    else:
        logscale=True
    columnno=len(column)
    if columnno>0:
        casesdata[list(column)].plot(logy=logscale,colormap="tab20b", xlabel ='Months\n',ylabel ='Number of people\n', 
              title='Graph showing the impact of COVID-19 vaccinations in England\n\n\n')
        #casesdata.plot(subplots=True, figsize=(8,8),colormap="tab20b",logy=True)
        plt.show() 
    else: 
        print("(Press SHIFT to select more than one category)")

  
graph=wdg.interactive_output(cases_graph,{'column': series, 'scale': scale})
#casesdata.plot(subplots=True, figsize=(8,8),colormap="tab20b",logy=True)


#This changes the layout of the widgets alongside the graph.
#I set the widgets to be next to the graph instead of on top
controls=wdg.VBox([series, scale])
position = wdg.HBox([graph,controls])




def refresh_graph():
    
    current=series.value
    if current==series.options[0]:
        other=(series.options[1],)
    else:
        other=(series.options[0],)
    series.value=other # forces the redraw
    series.value=current # now we can change it back



display(position)
plt.show()


wdg.HTML(value="<b>This data is updated regularly. Please check you are viewing the most up-to-date version</b>",placeholder='Some HTML',description='',
)

HTML(value='<b>This data is updated regularly. Please check you are viewing the most up-to-date version</b>', …

**Author and Copyright Notice:**  Data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*